In [1]:
# imports
import pandas as pd
import numpy as np
from sklearn import preprocessing
from scipy.stats import zscore
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import statsmodels.regression.linear_model as sm
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

In [2]:
# reading the csv file
data = pd.read_csv('winequality-white.csv',sep=';')
data2 = pd.read_csv('Complex9_GN32.csv')
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [3]:
# set the names for dataframe
x = data[['fixed acidity', 'volatile acidity','citric acid','residual sugar','chlorides','free sulfur dioxide',      
        'total sulfur dioxide','density','pH','sulphates','alcohol']]
#find the zscore and normalize
x.apply(zscore)
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
dataset = pd.DataFrame(x_scaled)
# Leaving the qualiity as is
dataset['quality'] = data['quality']
dataset.rename(columns={'0':'fixed acidity', '1':'volatile acidity','2':'citric acid','3':'residual sugar',
                        '4':'chlorides','5':'free sulfur dioxide','6':'total sulfur dioxide',
                        '7':'density','8':'pH','9':'sulphates','10':'alcohol'})
# making a new class and change the name according to the value of quality
dataset.loc[dataset['quality'] == 8, 'class'] = 'A'
dataset.loc[dataset['quality'] == 9, 'class'] = 'A'
dataset.loc[dataset['quality'] == 8, 'class'] = 'A'
dataset.loc[dataset['quality'] == 7, 'class'] = 'B'
dataset.loc[dataset['quality'] == 6, 'class'] = 'C'
dataset.loc[dataset['quality'] == 5, 'class'] = 'D'
dataset.loc[dataset['quality'] == 4, 'class'] = 'E'


In [15]:
# making the arrays to test functons
a = np.array([0,1,1,1,1,2,2,3]).reshape(-1,1)
b = np.array(['A','A','A','E','E','D','D','C'])

import scipy.stats
# finding entropy using scipy
def ent(data):
    data = data.value_counts()          
    entropy = scipy.stats.entropy(data) 
    return entropy

In [45]:
def entropy(a,b):
    
    out1 = len(a)
    # finding z score and classify outliers
    z= np.abs(zscore(a))
    a = a[(z < 1).all(axis=1)]
    out2 = len(a)
    # kmeans clustering on the cleaned data
    kmeans = KMeans(n_clusters=2, random_state=0).fit(a)
    pred = kmeans.predict(a)
    s1 = pd.Series(pred)
    # finding the percentage outlier
    pcentout = 1 - (out2/out1)
    print(ent(s1),pcentout)
    
entropy(a,b)

0.6365141682948128 0.25


In [ ]:
def ordinal_variation(a,b):
    return 0

In [80]:
def variance(a,b):
    # kmeans clustering
    kmean = KMeans(n_clusters=3,random_state=0).fit(a)
    pred = kmean.predict(a)
    # find the number of elements per cluster
    count = np.bincount(pred)
    var = 0
    variance = 0
    for cell in count:
        a = count
        b = len(pred)
        # finding the variance
        var += (np.var(pred))*(a/b)
    
    for cell in var:
        # summing array to find total variance
        variance = sum(var)
    return variance,print('No Outlier in KMeans')
variance(a,b)

No Outlier in KMeans


(1.3125, None)

In [50]:
def mdist(d):
    from scipy.spatial.distance import pdist
    # finding the zscore
    zs= np.abs(zscore(d))
    # find the manhatan distance using scipy
    zs = pdist(zs,'seuclidean',V=None)
    return zs
mdist(a)

array([2.00447929, 2.00447929, 2.00447929, 2.00447929, 1.50335947,
       1.50335947, 0.50111982, 0.        , 0.        , 0.        ,
       0.50111982, 0.50111982, 2.50559911, 0.        , 0.        ,
       0.50111982, 0.50111982, 2.50559911, 0.        , 0.50111982,
       0.50111982, 2.50559911, 0.50111982, 0.50111982, 2.50559911,
       0.        , 2.00447929, 2.00447929])

In [52]:
# naming columns
x = data[['fixed acidity', 'volatile acidity','citric acid','residual sugar','chlorides','free sulfur dioxide',
        'density','pH','sulphates','alcohol']]
y = data['quality']
# train and test data split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, shuffle= True)
# use multiple linear regression
Reg = LinearRegression()
Reg.fit(X_train, y_train)
# find the coeffcients
print('Coef: ', Reg.coef_)

Coef:  [ 1.64532048e-02 -1.93357158e+00  8.29245367e-02  6.45237585e-02
 -5.26632485e-01  3.36167542e-03 -1.10706905e+02  5.54174023e-01
  4.38373965e-01  2.46761028e-01]


In [7]:
# cleaner linear model using least squares
model = sm.OLS(y,x).fit()
print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:                quality   R-squared:                       0.984
Model:                            OLS   Adj. R-squared:                  0.984
Method:                 Least Squares   F-statistic:                 2.975e+04
Date:                Mon, 17 Jun 2019   Prob (F-statistic):               0.00
Time:                        13:39:17   Log-Likelihood:                -5578.3
No. Observations:                4898   AIC:                         1.118e+04
Df Residuals:                    4888   BIC:                         1.124e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
fixed acidity          -0.0555    